In [18]:
import numpy as np
import os
import shutil
import pandas as pd
import geojson
from json import JSONDecodeError

In [19]:
#PATH_output = '/media/user/Seagate_Ligue_DNA2/OV_TCGA_LYON/'
PATH_root = '/media/user/Watson/data_Lyon/Ovaires/'
PATH_mask = PATH_root+'3_Hovernet/'

In [24]:
PATH_patches = PATH_root+'1_c_patches_after_cleaning/' #à remplacer par ton dossier contenant les patches
PATH_hovernet = PATH_root+'3_Hovernet/' #à remplacer par ton dossier contenant l'output d'hovernet

if not os.path.exists(PATH_root+'4_Athena') :
    os.mkdir(PATH_root+'4_Athena')
    
    
PATH_output_df_count_cells = PATH_root+'4_Athena/1_df_count_cells/'
if not os.path.exists(PATH_output_df_count_cells) :
    os.mkdir(PATH_output_df_count_cells)
    
PATH_df_patch_select = PATH_root+'4_Athena/4_df_pyradiomics/'
if not os.path.exists(PATH_df_patch_select) :
    os.mkdir(PATH_df_patch_select)
    
PATH_patch_select = PATH_root+'4_Athena/1_patches_select/'
if not os.path.exists(PATH_patch_select) :
    os.mkdir(PATH_patch_select)
    
PATH_hovernet_select_json = PATH_root+'4_Athena/3_hovernet_select/'
if not os.path.exists(PATH_root+'4_Athena/3_hovernet_select/') :
    os.mkdir(PATH_root+'4_Athena/3_hovernet_select/')
    
PATH_hovernet_select_json = PATH_root+'4_Athena/3_hovernet_select/json/'
if not os.path.exists(PATH_hovernet_select_json) :
    os.mkdir(PATH_hovernet_select_json)
    
PATH_hovernet_select_overlay = PATH_root+'4_Athena/3_hovernet_select/overlay/'
if not os.path.exists(PATH_hovernet_select_overlay) :
    os.mkdir(PATH_hovernet_select_overlay)

In [21]:
patients = os.listdir(PATH_patches)
len(patients)

28

# df per patient creation

In [2]:
for patient in patients:
    print(patient)
    if not os.path.exists(PATH_output_df_count_cells+patient+'.csv'):
        df = pd.DataFrame()
        patches = os.listdir(PATH_patches+patient)
        for patch in patches:
            #print(patch)
            PATH_tot = PATH_mask+patient+'/json/'+patch[:-4]+'.json'
            PATH_overlay = PATH_mask+patient+'/overlay/'+patch[:-4]+'.png'

            if os.path.exists(PATH_tot) and os.path.exists(PATH_overlay) :
                with open(PATH_tot) as f :
                    nuclei = geojson.load(f)
                    nuclei = nuclei['nuc']
                count_tum = 0
                count_inf = 0
                count_others = 0
                count_connective = 0
                count_dead = 0
                count_nne = 0
                for key in list(nuclei.keys()) :
                   # print(nuclei[key]['type'])
                    if nuclei[key]['type'] == 1 :
                        count_tum += 1
                    elif nuclei[key]['type'] == 2 :
                        count_inf += 1  
                    elif nuclei[key]['type'] == 0 :
                        count_others += 1  
                    elif nuclei[key]['type'] == 3 :
                        count_connective += 1  
                    elif nuclei[key]['type'] == 4 :
                        count_dead += 1  
                    elif nuclei[key]['type'] == 5 :
                        count_nne += 1  
               # print('count_tum', count_tum)
            df_temp = pd.DataFrame({'patch': [patch[:-4]], 'count_cells': [len(nuclei.keys())], 'count_tum' : [count_tum], 'count_inf':[count_inf],'count_others':[count_others],'count_dead':[count_dead],'count_connective':[count_connective],'count_nne':[count_nne]})
            df = pd.concat([df, df_temp])
        df.to_csv(PATH_output_df_count_cells+patient+'.csv')
        print(len(df)) 

# Select patches

In [41]:
PATH_patches = PATH_root+'1_c_patches_after_cleaning/' #à remplacer par ton dossier contenant les patches
PATH_hovernet = PATH_root+'3_Hovernet/' #à remplacer par ton dossier contenant l'output d'hovernet

if not os.path.exists(PATH_root+'4_Athena_select_5000_noyaux') :
    os.mkdir(PATH_root+'4_Athena_select_5000_noyaux')
    
    
PATH_output_df_count_cells = PATH_root+'4_Athena_select_5000_noyaux/1_df_count_cells/'
if not os.path.exists(PATH_output_df_count_cells) :
    os.mkdir(PATH_output_df_count_cells)
    
PATH_df_patch_select = PATH_root+'4_Athena_select_5000_noyaux/1_df_patches_select/'
if not os.path.exists(PATH_df_patch_select) :
    os.mkdir(PATH_df_patch_select)
    
PATH_patch_select = PATH_root+'4_Athena_select_5000_noyaux/1_patches_select/'
if not os.path.exists(PATH_patch_select) :
    os.mkdir(PATH_patch_select)
    
PATH_hovernet_select_json = PATH_root+'4_Athena_select_5000_noyaux/3_hovernet_select/'
if not os.path.exists(PATH_root+'4_Athena_select_5000_noyaux/3_hovernet_select/') :
    os.mkdir(PATH_root+'4_Athena_select_5000_noyaux/3_hovernet_select/')
    
PATH_hovernet_select_json = PATH_root+'4_Athena_select_5000_noyaux/3_hovernet_select/json/'
if not os.path.exists(PATH_hovernet_select_json) :
    os.mkdir(PATH_hovernet_select_json)
    
PATH_hovernet_select_overlay = PATH_root+'4_Athena_select_5000_noyaux/3_hovernet_select/overlay/'
if not os.path.exists(PATH_hovernet_select_overlay) :
    os.mkdir(PATH_hovernet_select_overlay)

In [1]:
for patient in patients:
    print(patient)
    df = pd.read_csv('/media/user/Watson/data_Lyon/Ovaires/4_Athena/1_df_count_cells/'+patient+'.csv')
    df.sort_values(by='count_cells', ascending=False,inplace=True)
    i = 1
    while df[:i]['count_cells'].sum() < 5000 :
        print(i)
        i+=1
    print(i)
    df_select = df[:i]
    patches_select = list(df_select['patch'])
    print(len(patches_select))
    for patch in patches_select :

        if not os.path.exists(PATH_patch_select+patient) :
            os.mkdir(PATH_patch_select+patient)
        shutil.copy(PATH_patches+patient+'/'+patch+'.jpg', PATH_patch_select+patient+'/'+patch+'.jpg')
        if not os.path.exists(PATH_hovernet_select_overlay+patient) :
            os.mkdir(PATH_hovernet_select_overlay+patient)
        shutil.copy(PATH_hovernet+patient+'/overlay/'+patch+'.png', PATH_hovernet_select_overlay+patient+'/'+patch+'.png')
        if not os.path.exists(PATH_hovernet_select_json+patient) :
            os.mkdir(PATH_hovernet_select_json+patient)
        shutil.copy(PATH_hovernet+patient+'/json/'+patch+'.json', PATH_hovernet_select_json+patient+'/'+patch+'.json')
